In [6]:
!pip install BeautifulSoup4

    100% |████████████████████████████████| 92kB 2.3MB/s ta 0:00:011
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [7]:
!pip install lxml

    100% |████████████████████████████████| 5.9MB 111kB/s eta 0:00:01
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
from bs4 import BeautifulSoup
import csv
import requests
import lxml
from datetime import datetime, timedelta
import urllib

In [9]:
RU_MONTH_VALUES = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12',
}


def int_value_from_ru_month(date_str):
    for k, v in RU_MONTH_VALUES.items():
        date_str = date_str.replace(k, str(v))
    return date_str

In [26]:
days = timedelta(100)
deadline = datetime.now()-days
# proxies = {
#     'https': 'https://'+urllib.parse.quote('CHTPZ0\Anastasiya.Mittseva')+':'+\
#     urllib.parse.quote('Lifeiswonderful28')+'@'+urllib.parse.quote('proxy.lan')+':8080',
#     'http': 'http://'+urllib.parse.quote('CHTPZ0\Anastasiya.Mittseva')+':'+\
#     urllib.parse.quote('Lifeiswonderful28')+'@'+urllib.parse.quote('proxy.lan')+':8080',
# #     'http': 'http://CHTPZ0/Yuliya.Lapteva:пароль@10.74.1.16:8080',
# }   
   
def get_html(url):
    r = requests.get(url)
    return r.text
 
def get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find('div', class_='navigation')
    pages = divs.find_all('a', class_='a2')[-1].get('href')
    total_pages = pages.split('=')[1]
    return int(total_pages)

 
def get_page_data(html):
    
    soup = BeautifulSoup(html, 'lxml')
    news = soup.find_all('li', class_='news_list_item')
    for new in news:
        name = new.find('div', class_='news_list-ttl').getText()
        url = "http://minpromtorg.gov.ru" + new.find('a').get('href')
        date_str = new.find('span', class_='main-block-date').getText()
        date = int_value_from_ru_month(date_str.lower())
        
        date = datetime.strptime(date, '%d %m %Y')
        
        if date > deadline:
#             print(date, deadline)
            if "конкурс" in str(name).lower():
                data = {'name':name,
                        'url':url,
                        'date':date_str}
                print(data)
        else:
            return False
    return True
 
 
def main():
    base_url = "http://minpromtorg.gov.ru/press-centre/news/"
    page_part = "?from_18=&from_18="
 
    total_pages = get_total_pages(get_html(base_url))
    print(total_pages)
    fl = True
    for i in range(1, total_pages):
        url_gen = base_url + page_part + str(i)
#         print(url_gen)
        html = get_html(url_gen)
        fl = get_page_data(html)
        if fl==False:
            break
#         print(i)

 
if __name__ == '__main__':
    main()

543
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=1
1
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=2
2
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=3
3
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=4
{'name': ' Открыт прием заявок на Конкурс биомедицинских проектов «Стартап-ралли 2018», организованный при поддержке Минпромторга России ', 'url': 'http://minpromtorg.gov.ru/press-centre/news/#!otkryt_priem_zayavok_na_konkurs_biomedicinskih_proektov_startapralli_2018_organizovannyy_pri_podderzhke_minpromtorga_rossii', 'date': '13 июня 2018'}
4
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=5
5
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=6
6
http://minpromtorg.gov.ru/press-centre/news/?from_18=&from_18=7
{'name': ' Минпромторг России приглашает к участию в специальной номинации конкурса «Проектный Олимп» ', 'url': 'http://minpromtorg.gov.ru/press-centre/news/#!minpromtorg_rossii_priglashaet_k_